In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

c:\Users\evann\anaconda3\envs\asr\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv('../asr-train/cv-valid-dev_with_predicted_transcriptions.csv')

In [4]:
tokenizer = AutoTokenizer.from_pretrained("hkunlp/instructor-large")
model = AutoModel.from_pretrained("hkunlp/instructor-large")

Some weights of T5Model were not initialized from the model checkpoint at hkunlp/instructor-large and are newly initialized: ['decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.0.SelfAttention.o.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'deco

In [ ]:
# Function to extract words/phrases from a sentence
def extract_phrases(sentence, ngram_range=(1, 2)):
    # CountVectorizer to extract 1-gram and 2-gram phrases (Assuming phrases are 2 words)
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    ngrams = vectorizer.fit_transform([sentence])
    feature_names = vectorizer.get_feature_names_out()

    return feature_names


# Function to compute similarity between a word/phrase and hot words
def compute_similarity(phrase, hot_words):
    # Encode the phrase and hot words using the tokenizer and model
    phrase_tokens = tokenizer([phrase], return_tensors="pt", padding=True, truncation=True)
    hotword_tokens = tokenizer(hot_words, return_tensors="pt", padding=True, truncation=True)
    
    # embed the phrase and hot words using the model
    with torch.no_grad():
        phrase_embedding = model.encoder(**phrase_tokens).last_hidden_state.mean(dim=1).numpy()
        hotword_embeddings = model.encoder(**hotword_tokens).last_hidden_state.mean(dim=1).numpy()
    
    # Compute cosine similarity between the phrase and each hot word
    similarities = cosine_similarity(phrase_embedding, hotword_embeddings)
    
    # Return True if the maximum similarity is above 0.7 (similarity threshold)
    return np.max(similarities) > 0.7 


# Apply the similarity check to the 'predicted_transcription' from fine-tuned model
def check_similarity_in_sentence(sentence, hot_words):
    phrases = extract_phrases(sentence)
    
    # Check similarity for each phrase in the sentence
    for phrase in phrases:
        if compute_similarity(phrase, hot_words):
            return True
    
    return False

In [ ]:
hotwords = ['be careful', 'destroy', 'stranger']

# Test on a subset of the dataset
df['predicted_transcription'] = df['predicted_transcription'].str.lower()

# Apply the similarity check for each sentence in the DataFrame
df['similarity'] = df['text'].apply(lambda x: check_similarity_in_sentence(x, hotwords))

# Save the updated DataFrame
output_file_path = 'cv-valid-dev-with-similarity.csv'  # Output file name
df.to_csv(output_file_path, index=False)

print(f"Updated dataset saved to {output_file_path}")


C:\Users\evann\AppData\Local\Temp\ipykernel_6444\162227799.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['generated_text'] = df_test['generated_text'].str.lower()


Updated dataset saved to cv-valid-dev-with-similarity.csv


C:\Users\evann\AppData\Local\Temp\ipykernel_6444\162227799.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['similarity'] = df_test['text'].apply(lambda x: check_similarity_in_sentence(x, hotwords))
